# Model Training
The following notebook trains two deep learning models:
1. **SheetClassifier**: A model for identifying images of metal sheets with defects, classifying them by their defect.
2. **SheetHighlighter**: A model which takes images of metal sheets with defects and highlights the defects in the image.

In [111]:
# Imports
%load_ext autoreload
%autoreload 2

import torch
from torch import nn
from torch.utils.data import DataLoader, Dataset, random_split
import matplotlib.pyplot as plt

import pandas as pd
import numpy as np

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [119]:
class Trainer():
    def __init__(self, epochs:int, learning_rate:float, model_type:nn.Module, dataset:Dataset, batch_size:int = 10, criterion = nn.CrossEntropyLoss, show:bool = False):
        self.epochs, self.lr, self.model_type, self.dataset, self.batch_size, self.criterion_type, self.show = epochs, learning_rate, model_type, dataset, batch_size, criterion, show
        self.create_dataloaders()

    def run_experiment(self, model = None):
        # Runs the experiment with the given criteria, returning a trained model
        #training_losses = [] # For graphing, should we choose

        self.model = self.model_type() if not model else model
        self.has_acc = hasattr(self.model, 'get_accuracy')
        self.criterion = self.criterion_type()
        self.optimizer = torch.optim.Adam(self.model.parameters(), lr=self.lr)

        for epoch in range(self.epochs):
            self.model.train()
            running_loss = 0.0
            running_acc = 0.0
            for i, (x,y) in enumerate(self.training_loader):
                self.optimizer.zero_grad()
                yhat = self.model(x)
                loss = self.criterion(yhat, y)
                loss.backward()
                self.optimizer.step()
                running_loss += loss.item()
                if self.has_acc: running_acc += self.model.get_accuracy(yhat, y)
            training_acc = running_acc / len(self.training_loader)
            training_loss = running_loss / len(self.training_loader)
            if (epoch % 10 == 0 or training_acc >= .99) and self.show:
                msg = f'Epoch [{epoch + 1}/{self.epochs}], Train Loss: {training_loss:.2f}'
                if self.has_acc: msg+=f', Accuracy: {training_acc:.2f}'
                print(msg)
                if training_acc >= .99: break
        if self.show: print('Experiment Complete')
        return self.model

    def evaluate_model(self, model:nn.Module = None, loader = None):
        # Evaluates the trained model
        if not loader: loader = self.testing_loader
        if not model: model = self.model
        has_acc = hasattr(model, 'get_accuracy')
        criterion = self.criterion_type()

        model.eval()
        running_loss = 0.0
        running_acc = 0.0
        for i, (x,y) in enumerate(loader):
            yhat = model(x)
            loss = criterion(yhat, y)
            running_loss += loss.item()
            if has_acc: running_acc += model.get_accuracy(yhat, y)
        msg = f'[Evaluation over {len(loader)} Batches], Test Loss: {running_loss/len(loader):.2f}'
        if has_acc: msg+=f', Accuracy: {running_acc/len(loader):.2f}'
        print(msg)
        return model
    
    def create_dataloaders(self):
        num_items = len(self.dataset)
        num_train = round(num_items * 0.8)
        num_val = num_items - num_train
        
        self.training_dataset, self.testing_dataset = random_split(self.dataset, [num_train, num_val])
        self.training_loader = DataLoader(self.training_dataset,  batch_size=self.batch_size, shuffle=True)
        self.testing_loader = DataLoader(self.testing_dataset, batch_size=self.batch_size, shuffle=False)
         
    def calculate_accuracy(self, predictions:list, true_y:list):
        accuracy_df = pd.DataFrame(zip(predictions, true_y),columns=['predictions','y_values'])
        accuracy_df.predictions = accuracy_df.predictions >= 0.5
        accuracy_df.y_values = accuracy_df.y_values == 1
        accuracy_df = accuracy_df.predictions == accuracy_df.y_values
        accuracy = len(accuracy_df[accuracy_df])/len(accuracy_df)
        
        if self.show: print('Our Model Accuracy', accuracy)
        return accuracy

# Training the Classifier

In [133]:
from models.SheetClassifier import SheetClassifier
from datasets.ClassificationSet import ClassificationSet

class_set = ClassificationSet('/home/sms/Documents/hobbies/projects/ml_data/metal_sheets')
trainer = Trainer(10, 0.003, SheetClassifier, class_set, 10, criterion=nn.CrossEntropyLoss, show=True)
sc = SheetClassifier()

### Pre-Training Predictions

In [134]:
trainer.evaluate_model(sc)

[Evaluation over 60 Batches], Test Loss: 4.96, Accuracy: 0.31


SheetClassifier(
  (model): Sequential(
    (0): Conv2d(1, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): BatchNorm2d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): Conv2d(8, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): ReLU()
    (5): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (lin1): Linear(in_features=65536, out_features=3, bias=True)
)

In [135]:
sc = trainer.run_experiment()

Epoch [1/10], Train Loss: 5.13, Accuracy: 0.63
Epoch [2/10], Train Loss: 1.06, Accuracy: 0.88
Epoch [3/10], Train Loss: 0.26, Accuracy: 0.96
Epoch [4/10], Train Loss: 0.17, Accuracy: 0.97
Epoch [5/10], Train Loss: 0.10, Accuracy: 0.98
Epoch [6/10], Train Loss: 0.05, Accuracy: 0.99
Epoch [7/10], Train Loss: 0.05, Accuracy: 0.99
Epoch [8/10], Train Loss: 0.06, Accuracy: 0.98
Epoch [9/10], Train Loss: 0.06, Accuracy: 0.99
Epoch [10/10], Train Loss: 0.11, Accuracy: 0.98
Experiment Complete


In [136]:
trainer.evaluate_model(sc)

[Evaluation over 60 Batches], Test Loss: 4.69, Accuracy: 0.73


SheetClassifier(
  (model): Sequential(
    (0): Conv2d(1, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): BatchNorm2d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): Conv2d(8, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): ReLU()
    (5): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (lin1): Linear(in_features=65536, out_features=3, bias=True)
)

# Training the Highlighter

In [139]:
from models.SheetHighlighter import SheetHighlighter
from datasets.HighlighterSet import HighlighterSet

highlight_set = HighlighterSet('/home/sms/Documents/hobbies/projects/ml_data/metal_sheets')
trainer = Trainer(11, 0.003, SheetHighlighter, highlight_set, 10, criterion=nn.BCELoss, show=True)
sh = SheetHighlighter()

In [138]:
trainer.evaluate_model(sh)

[Evaluation over 60 Batches], Test Loss: 1.33, Accuracy: 0.42


SheetHighlighter(
  (model): Sequential(
    (0): Conv2d(1, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): BatchNorm2d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): Conv2d(8, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): ReLU()
    (5): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ConvTranspose2d(16, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU()
    (8): Conv2d(8, 1, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): Sigmoid()
  )
)

In [140]:
sh = trainer.run_experiment()

Epoch [1/11], Train Loss: 0.05, Accuracy: 0.98
Epoch [2/11], Train Loss: 0.03, Accuracy: 0.99
Experiment Complete


In [141]:
trainer.evaluate_model(sh)

[Evaluation over 60 Batches], Test Loss: 0.02, Accuracy: 0.99


SheetHighlighter(
  (model): Sequential(
    (0): Conv2d(1, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): BatchNorm2d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): Conv2d(8, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): ReLU()
    (5): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ConvTranspose2d(16, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU()
    (8): Conv2d(8, 1, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): Sigmoid()
  )
)